In [41]:
import os
from dotenv import load_dotenv
from langchain import hub
from langchain import OpenAI, LLMChain
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents import create_tool_calling_agent, initialize_agent, load_tools
from langchain.agents import AgentExecutor, ZeroShotAgent, LLMSingleActionAgent, AgentOutputParser
from langchain.memory import ConversationBufferMemory
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.google_finance import GoogleFinanceQueryRun
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_community.utilities.google_finance import GoogleFinanceAPIWrapper
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool

from fastapi import FastAPI
from langserve import add_routes
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.messages import BaseMessage
from typing import List
import uvicorn

In [42]:
load_dotenv()

## openai api
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

## google api key
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY") # note: the api key should be doubles quotes. i.e., ""
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY") # google search api
os.environ["GOOGLE_CSE_ID"] = os.getenv("GOOGLE_CSE_ID") # google custom search engine id
## Langsmith tracking
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") # where monitoring results needs to be stored

## tavily api
os.environ['TAVILY_API_KEY']= os.getenv("TAVILY_API_KEY")

In [61]:
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_community.tools.google_finance import GoogleFinanceQueryRun
from langchain_community.utilities.google_finance import GoogleFinanceAPIWrapper
search = GoogleFinanceQueryRun(api_wrapper=GoogleFinanceAPIWrapper())
google_finance_tool = Tool(
  name="google_finance",
    description="Search Google for recent financial results.",
    func=search.run,  
)


In [62]:
# search tool using Google
search = GoogleSearchAPIWrapper()
google_search_tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)


## Tools
tools=[google_search_tool, google_finance_tool] # google_tool, google_search_tool, "google-finance"

In [70]:
# Set up the prompt with input variables for tools, user input and a scratchpad for the model to record its workings
template = """
You are an Stock Trading Expert. 
Consider the folllowing metrics to screen the different stocks:
Small CAP company only



You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a stock trader when giving your final answer and list the names of the stocks.

Question: {input}
{agent_scratchpad}"""

In [71]:
# Set up a prompt template
from langchain.prompts import BaseChatPromptTemplate
from langchain.schema import AgentAction, AgentFinish, HumanMessage, SystemMessage

class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
            
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]
    
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [72]:
import re
from typing import List, Union

class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        
        # Parse out the action and action input
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        
        # If it can't parse the output it raises an error
        # You can add your own logic here to handle errors in a different way i.e. pass to a human, give a canned response
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
    
output_parser = CustomOutputParser()

In [73]:
# Initiate our LLM - default is 'gpt-3.5-turbo'
llm = ChatOpenAI(temperature=0)

# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Using tools, the LLM chain and output_parser to make an agent
tool_names = [tool.name for tool in tools]

In [74]:
# creating agent
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    # We use "Observation" as our stop sequence so it will stop when it receives Tool output
    # If you change your prompt template you'll need to adjust this as well
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [75]:
# Initiate the agent that will respond to our queries
# Set verbose=True to share the CoT reasoning the LLM goes through
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [76]:
agent_executor.run("Top five stocks to buy in 2024?")



> Entering new AgentExecutor chain...
Thought: Since we are looking for small CAP companies, we need to focus on companies with a market capitalization between $300 million and $2 billion.
Action: google_search
Action Input: "top small cap stocks 2024"

Observation:Apr 2, 2024 ... Best Small-Cap Stocks of April 2024 · ACM Research, Inc. (ACMR) · Titan Machinery Inc. (TITN) · Stride, Inc. (LRN) · Digi International Inc. (DGII). Mar 13, 2024 ... 9 Best Small-Cap Stocks to Buy for 2024 · AppFolio Inc. (APPF) · BellRing Brands Inc. (BRBR) · Eagle Materials Inc. (EXP) · Guidewire Software ... Feb 2, 2024 ... 10 of the Best Small-Cap Stocks to Buy Before They Rebound · Lithium Americas (Argentina) LAAC · Hanesbrands HBI · VF Corp VFC · Arcadium Lithium ... Feb 3, 2024 ... Small Size, Big Impact: 6 Mighty Small-Cap Value Stocks For 2024 · 6 Best Small-Cap Value Stocks · 1. Everi Holdings (EVRI) · 2. Olaplex ... Apr 2, 2024 ... Best small-cap stocks, ordered by one-year performance ; SWVL. S

'The top five small-cap stocks to buy in 2024 are ACM Research, Inc. (ACMR), Titan Machinery Inc. (TITN), Stride, Inc. (LRN), Digi International Inc. (DGII), and AppFolio Inc. (APPF).'

## LLM Agent with History

In [77]:
# Set up a prompt template which can interpolate the history
template_with_history = """
You are an Stock Trading Expert. 
Consider the folllowing metrics to screen the different stocks:
1. Small CAP company

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a stock trader and give the names of the stocks.

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [78]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # The history template includes "history" as an input variable so we can interpolate it into the prompt
    input_variables=["input", "intermediate_steps", "history"]
)

llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [79]:
# Conversational memory
from langchain.memory import ConversationBufferWindowMemory
# Initiate the memory with k=2 to keep the last two turns
# Provide the memory to the agent
memory = ConversationBufferWindowMemory(k=2)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)


In [80]:
agent_executor.run("Top five stocks to buy in 2024?")



> Entering new AgentExecutor chain...
Question: Top five stocks to buy in 2024?
Thought: I should look for companies with strong growth potential and solid financials.
Action: google_search
Action Input: "top growth stocks 2024"

Observation:3 days ago ... 10 Best Growth Stocks to Buy for 2024 · Nvidia Corp. (NVDA) · Alphabet Inc. (GOOG, GOOGL) · Meta Platforms Inc. (META) · JPMorgan Chase & Co. (JPM). Jan 22, 2024 ... 3 Top Growth Stocks to Buy and Hold for 2024 · Rentokil Initial RTO · AstraZeneca AZN · Tyler Technologies TYL. The first stock on our list of ... 5 days ago ... (NASDAQ:AMD) and the electric vehicle manufacturer Tesla, Inc. (NASDAQ:TSLA). AMD's shares are up by a whopping 445% over the past five years, ... Feb 6, 2024 ... 10 Best Growth Stocks to Buy for the Long Term · Rentokil Initial RTO · AstraZeneca AZN · Tyler Technologies TYL · Coloplast CLPBY · Airbus EADSY ... Apr 1, 2024 ... Growth stocks thrive during economic expansions when interest rates are low—here are

'The top five stocks to buy in 2024 are Nvidia Corp. (NVDA), Alphabet Inc. (GOOG, GOOGL), Meta Platforms Inc. (META), JPMorgan Chase & Co. (JPM), and Rentokil Initial RTO.'

In [81]:
agent_executor.run("are these small cap companies?")



> Entering new AgentExecutor chain...
Question: are these small cap companies?
Thought: I should check the market capitalization of each of these companies to determine if they are small cap.
Action: google_search
Action Input: "Nvidia Corp market capitalization"

Observation:As of April 2024 NVIDIA has a market cap of $2.193 Trillion. This makes NVIDIA the world's third most valuable company by market cap according to our data. The ... Find the latest NVIDIA Corporation (NVDA) stock quote, history, news and other vital information to help you with your stock trading and investing. Ranking the world's top companies by market cap, market value, revenue and many more metrics. Mar 7, 2024 ... Nvidia shares rose more than 4% on Thursday, bringing its market capitalization to more than $2.28 trillion, an increase of over $1 trillion ... NVIDIA Market Cap: 2.193T for April 26, 2024 · Market Cap Chart · Historical Market Cap Data · Market Capitalization Definition · Market Cap Range, Past 5

'None of the top five stocks to buy in 2024 (Nvidia Corp., Alphabet Inc., Meta Platforms Inc., JPMorgan Chase & Co., Rentokil Initial) are small cap companies.'

In [82]:
agent_executor.run("give me small cap companies to buy in 2024?")



> Entering new AgentExecutor chain...
Question: give me small cap companies to buy in 2024?
Thought: I need to find small cap companies to recommend for buying in 2024.
Action: google_search
Action Input: "top small cap companies to buy in 2024"

Observation:Mar 13, 2024 ... 9 Best Small-Cap Stocks to Buy for 2024 · AppFolio Inc. (APPF) · BellRing Brands Inc. (BRBR) · Eagle Materials Inc. (EXP) · Guidewire Software ... Apr 2, 2024 ... Best Small-Cap Stocks of April 2024 · ACM Research, Inc. (ACMR) · Titan Machinery Inc. (TITN) · Stride, Inc. (LRN) · Digi International Inc. (DGII). Dec 14, 2023 ... The Street's best small-caps stocks ; Flywire, FLYW, Payment processing services, 1.31, Strong Buy ; Academy Sports and Outdoors, ASO, Specialty ... Feb 3, 2024 ... 6 Best Small-Cap Value Stocks · 1. Everi Holdings (EVRI) · 2. Olaplex Holdings (OLPX) · 3. American Vanguard (AVD) · 4. Hudson Technologies (HDSN). Feb 2, 2024 ... Lithium Americas (Argentina) tops our list of the best small-cap

'Some small cap companies to consider buying in 2024 are ACM Research, Inc. (ACMR), Titan Machinery Inc. (TITN), Stride, Inc. (LRN), Digi International Inc. (DGII), Everi Holdings (EVRI), Olaplex Holdings (OLPX), American Vanguard (AVD), Hudson Technologies (HDSN), Flywire (FLYW), Academy Sports and Outdoors (ASO), Lithium Americas (Argentina), Aehr Test Systems (AEHR), Enovix (ENVX), and Hims & Hers Health (HIMS).'

In [83]:
agent_executor.run("which of these companies has an Earnings Growth Potential of 20 percent or more in the coming years.?")



> Entering new AgentExecutor chain...
Question: which of these companies has an Earnings Growth Potential of 20 percent or more in the coming years?
Thought: Let's search for the earnings growth potential of each of these small cap companies.
Action: google_search
Action Input: "ACM Research Inc earnings growth potential"

Observation:Mar 27, 2024 ... ... company's earnings growth potential. Generally speaking, other things being equal, firms with a high return on equity and profit retention ... Fair Value is the appropriate price for the shares of a company, based on its earnings and growth rate also interpreted as when P/E Ratio = Growth Rate. Oct 28, 2023 ... It might be of some concern to shareholders to see the ACM Research, Inc. ( NASDAQ:ACMR ) share price down 25% in the... Oct 11, 2023 ... ... ACM Research Inc is a US-based company ... ACMR's growth rank of 10/10 suggests strong growth potential. The company's 3-Year Revenue Growth Rate ... Jan 15, 2024 ... Earnings growth is

'Among the small cap companies mentioned, Stride Inc (LRN) and Academy Sports and Outdoors (ASO) are expected to have earnings growth potential of 20 percent or more in the coming years.'

In [84]:
agent_executor.run("what about Canadian stocks with the same criterias?")



> Entering new AgentExecutor chain...
Question: what about Canadian stocks with the same criterias?
Thought: Let's search for Canadian small cap companies with potential earnings growth.
Action: google_search
Action Input: Canadian small cap companies with potential earnings growth

Observation:Apr 2, 2024 ... Of course, most companies start out as small-caps, but by continually growing their earnings, their share prices appreciate. This can increase ... 7 days ago ... ... earnings growth outlook. And we have leaned against small cap stocks whose earnings are at greater risk from higher rates. Earnings face a ... Investment Objective. The fund seeks to provide long-term capital appreciation by investing in mid-cap stocks with potential for above-average earnings growth ... Apr 12, 2023 ... ... small-cap stocks that are not considered penny stocks. ... Canadian oil and gas development and production company. ... earnings-per-share (EPS) ... Feb 14, 2023 ... Source: RBC Capital. Source

'There are Canadian small cap companies with potential earnings growth, but specific names were not provided in the search results. Further research or analysis may be needed to identify specific Canadian small cap stocks meeting the criteria.'

In [86]:
agent_executor.run("what about the previous stocks that has a Return on Equity (ROE) of around 20-30 percent? To calculate ROE, divide a company's net annual income by its shareholders' equity.")



> Entering new AgentExecutor chain...
Question: what about the previous stocks that has a Return on Equity (ROE) of around 20-30 percent?
Thought: Let's calculate the Return on Equity for Stride Inc (LRN) and Academy Sports and Outdoors (ASO) to see if they fall within the 20-30 percent range.
Action: google_search
Action Input: "Stride Inc ROE calculation"

Observation:Current Ratio · Quick Ratio · Debt/Equity Ratio · ROE · ROA · ROI · Return Tang ... Return on equity measures a corporation's ... Stride Inc. is a premier provider ... Apr 17, 2024 ... How To Calculate Return On Equity? The formula for ROE is: Return on Equity = Net Profit (from continuing operations) ÷ Shareholders' Equity. So ... Feb 11, 2024 ... How To Calculate Return On Equity? · What Is The Relationship Between ROE And Earnings Growth? · A Side By Side comparison of Stride's Earnings ... Mar 14, 2024 ... How Is ROE Calculated? ROE can be calculated by using the formula: Return on Equity = Net Profit (from contin

'Academy Sports and Outdoors (ASO) has a Return on Equity (ROE) of around 20-30 percent.'